# Forced Photometry Light Curve

<br>Owner: **Justin Myles** ([@jtmyles](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@jtmyles))
<br>Last Verified to Run: **N/A -- in development**

This project addresses issue [#63: HSC Re-run](https://github.com/LSSTScienceCollaborations/StackClub/issues/63)

This notebook demonstrates the [LSST Science Piplines data processing tutorial](https://pipelines.lsst.io/) with emphasis on how a given [obs package](https://github.com/lsst/obs_base) works under the hood of the command line tasks. It makes use of the [obs_subaru](https://github.com/lsst/obs_subaru) package to measure a forced photometry light curve for a small patch in the HSC sky in the [ci_hsc](https://github.com/lsst/ci_hsc/) repository. 

### Learning Objectives:
After working through and studying this notebook you should be able to understand how to use the DRP pipeline from image visualization through to a forced photometry light curve. Specific learning objectives include: 
   1. [configure command-line tasks](https://pipelines.lsst.io/v/w-2018-12/modules/lsst.pipe.base/command-line-task-config-howto.html) for your science case
   2. TODO
   
Other techniques that are demonstrated, but not empasized, in this notebook are
   1. Use the `butler` to fetch data
   2. Visualize data with the LSST Stack
   3. TODO

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.


## Set Up

In [28]:
# TODO imports

## Part I: Interacting with data. Introduction to the Butler
https://pipelines.lsst.io/getting-started/data-setup.html

In [ ]:
!eups list lsst_distrib

In [ ]:
!mkdir -p DATA

In [ ]:
!pwd

In [ ]:
!setup -j -r /global/u2/j/jmyles/repos/ci_hsc

In [ ]:
!echo "$CI_HSC_DIR"

In [ ]:
# ingest raw data into Butler repo
!ingestImages.py DATA /global/u2/j/jmyles/repos/ci_hsc/raw/*.fits --mode=link

In [ ]:
!installTransmissionCurves.py DATA

In [ ]:
# injest calibration images into Butler repo
!ln -s /global/u2/j/jmyles/repos/ci_hsc/CALIB/ DATA/CALIB

In [ ]:
!mkdir -p DATA/ref_cats

In [ ]:
# injest reference catalog into Butler repo
!ln -s /global/u2/j/jmyles/repos/ci_hsc/ps1_pv3_3pi_20170110 DATA/ref_cats/ps1_pv3_3pi_20170110

# Part 2: Calibrating single frames
https://pipelines.lsst.io/getting-started/processccd.html

In [ ]:
# review what data will be processed
!processCcd.py DATA --rerun processCcdOutputs --id --show data
# id allows you to select data by data ID
# unspecified id selects all raw data
# example IDs: raw, filter, visit, ccd, field
# show data turns on dry-run mode

In [ ]:
!which processCcd.py

In [ ]:
!processCcd.py DATA --rerun processCcdOutputs --id
# all cl tasks write output datasets to a Butler repo
# --rerun configured to write to processCcdOutputs
# other option is --output

# Part 3: Displaying exposures and source tables output by processCcd.py
https://pipelines.lsst.io/getting-started/display.html

This part of the tutorial is omitted for now.

# Part 4: Coadding images
https://pipelines.lsst.io/getting-started/coaddition.html

* A sky map is a tiling of the celestial sphere. It is composed of one or more tracts.
* A tract is composed of one or more overlapping patches. Each tract has a WCS.

In [ ]:
# make a discrete sky map that covers the exposures that have already been processed
!makeDiscreteSkyMap.py DATA --id --rerun processCcdOutputs:coadd --config skyMap.projection="TAN"

# the configuration field specifies the WCS Projection
# one of the FITS WCS projection codes, such as:
#           - STG: stereographic projection
#           - MOL: Molleweide's projection
#           - TAN: tangent-plane projection